In [2]:
import re
import os
import sys
import math
import utils
import random
import layers
import dataset
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
from network import network

In [14]:
#read params
regex="^.*=(.*)$"
with open("train.params") as f:
    learning_rate=re.findall(regex,f.readline())[0]
    batch_size=re.findall(regex,f.readline())[0]
    epoch=re.findall(regex,f.readline())[0]
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))
model_dir="model_{}_{}_{}".format(learning_rate,batch_size,epoch)
os.makedirs(model_dir)

training with a learning rate of 0.1 for 3 epochs with batchs of size 10


In [13]:
net=network(learning_rate)
data_all=np.load("all_data.npy")  
label_all=np.load("all_labels.npy")

(100, 24, 137, 137, 4)
(100, 32, 32, 32)


In [ ]:
# setup training


# train network
print("starting training at {}".format(cur_time))
loss_session=[]
acc_session=[]
for e in range(epoch):
    perm=np.random.permutation(N)
    data_all=data_all[perm]
    label_all=label_all[perm]
    split_size=math.ceil(N/batch_size)
    data_batchs=np.array_split(data_all,split_size)
    label_batchs=np.array_split(label_all,split_size)
    loss_epoch=[]
    acc_epoch=[]
    # print("starting epoch_{:03d}".format(e))
    epoch_dir="{}/epoch_{:03d}".format(train_dir,e)
    os.makedirs(epoch_dir)
    
    batch_number=0
    for data,label in zip(data_batchs,label_batchs):
        batch_info=sess.run([net.mean_loss,net.mean_accuracy,net.optimizing_op],feed_dict={net.X:data, net.Y: label})
        loss_batch=batch_info[0]
        acc_batch=batch_info[1]
        loss_epoch.append(loss_batch)
        acc_epoch.append(acc_batch)
        batch_number+=1
        if batch_number%10==0:
            print("epoch_{:03d}-batch_{:03d}: loss={}, acc={}".format(e,batch_number,loss_batch,acc_batch))
    loss_session.append(loss_epoch)
    acc_session.append(acc_epoch)
    net.
    